# Recon3DplusthreeCompetition).json [Competitive mechanism only]

##Changing medium

In [1]:
import cobra
import pandas as pd
import numpy as np
import pickle
from cobra.flux_analysis import pfba

In [2]:
M = cobra.io.load_json_model('Recon3DplusthreeCompetitive.json')

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


## Tyrosine is not enough in the diet (model1)

In [3]:
from cobra.flux_analysis import pfba
# import map visualization tools
from utils import show_map
import escher
map_loc = 'Recon3DplusthreeCompetitiveMap.json' # the escher map used below
# show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

model=M.copy()

In [4]:
model1=model.copy()

In [5]:
phe_L_c_related=['lyspheile_TOBD','phetyr_TOBD','phetyrgln_TOBD','hisphearg_TOBD','valtrpphe_TOBD','phetyrlys_TOBD',
                    'glyphe_TOBD','lysvalphe_TOBD','thrphearg_TOBD','argleuphe_TOBD','tyralaphe_TOBD','cysaspphe_TOBD',
                    'argphearg_TOBD','glylysphe_TOBD','metphearg_TOBD','mettrpphe_TOBD','prophe_TOBD','pheasnmet_TOBD',
                    'pheasp_TOBD','pheglnphe_TOBD','pheleu_TOBD','pheleuasp_TOBD','pheleuhis_TOBD','tyrphetyr_TOBD',
                    'phelysala_TOBD','phelyspro_TOBD','tyrtrpphe_TOBD','asnpheasp_TOBD','asnphecys_TOBD','phepheasn_TOBD',
                    'asntyrphe_TOBD','serphelys_TOBD','pheproarg_TOBD','valleuphe_TOBD','phesertrp_TOBD','lysglnphe_TOBD',
                    'argalaphe_TOBD','phethrlys_TOBD','trpphe_TOBD','phetrpleu_TOBD','valphearg_TOBD','phephethr_TOBD',
                'phephe_TOBD','glyphe_TOBD','trpglyphe_TOBD']
for reaction in phe_L_c_related:
    model1.reactions.get_by_id(reaction).lower_bound=0
    model1.reactions.get_by_id(reaction).upper_bound=0

phe_L_c_related=['EX_lyspheile_e','EX_phetyr_e','EX_phetyrgln_e','EX_hisphearg_e','EX_valtrpphe_e','EX_phetyrlys_e',
                    'EX_glyphe_e','EX_lysvalphe_e','EX_thrphearg_e','EX_argleuphe_e','EX_tyralaphe_e','EX_cysaspphe_e',
                    'EX_argphearg_e','EX_glylysphe_e','EX_metphearg_e','EX_mettrpphe_e','EX_prophe_e','EX_pheasnmet_e',
                    'EX_pheasp_e','EX_pheglnphe_e','EX_pheleu_e','EX_pheleuasp_e','EX_pheleuhis_e','EX_tyrphetyr_e',
                    'EX_phelysala_e','EX_phelyspro_e','EX_tyrtrpphe_e','EX_asnpheasp_e','EX_asnphecys_e','EX_phepheasn_e',
                    'EX_asntyrphe_e','EX_serphelys_e','EX_pheproarg_e','EX_valleuphe_e','EX_phesertrp_e','EX_lysglnphe_e',
                    'EX_argalaphe_e','EX_phethrlys_e','EX_trpphe_e','EX_phetrpleu_e','EX_valphearg_e','EX_phephethr_e',
                'EX_phephe_e','EX_glyphe_e','EX_trpglyphe_e']
for reaction in phe_L_c_related:
    model1.reactions.get_by_id(reaction).lower_bound=0
    model1.reactions.get_by_id(reaction).upper_bound=0

In [6]:
# Setting objective function: biomass reaction in the brain + dopamine production + serotonin production
quadratic_objective = model1.problem.Objective(model1.reactions.biomass_reaction_INB.flux_expression+
                                              model1.reactions.get_by_id('3HLYTCL_INB').flux_expression+
                                              model1.reactions.get_by_id('5HLTDL_INB').flux_expression,
    direction='max')
model1.objective = quadratic_objective

In [7]:
# Changing the phe and tyr in the medium, because they also use for producing dopa.
model1.reactions.EX_phe_L_e.lower_bound=-0.2
model1.reactions.EX_phe_L_e.upper_bound=-0.2
model1.reactions.EX_tyr_L_e.lower_bound=-0.04

In [8]:
for rxn in model1.metabolites.phe_L_c.reactions:
    model1.reactions.get_by_id(rxn.id).upper_bound=0
    model1.reactions.PheTR_TOBD.upper_bound=1000
    model1.reactions.biomass_reaction.upper_bound=1000
    model1.reactions.r0399.upper_bound=1000
    model1.reactions.PHETHPTOX2.upper_bound=1000

In [9]:
## Other reactions to produce l-dopa is forbidded
model1.reactions.TYRDOPO_INB.upper_bound=0
model1.reactions.TYR3MO2_INB.upper_bound=0
## liver biomass is constant
model1.reactions.biomass_reaction.lower_bound=0.05
model1.reactions.biomass_reaction.upper_bound=0.05
## do not need too much biomass in the brain
model1.reactions.biomass_reaction_INB.upper_bound=0.1
# Phe secretion in the blood is blocked, otherwise there is no competition.
model1.reactions.EX_phe_L_BD.upper_bound=0
# No phenylpyruvate production in the brain and liver
model1.reactions.PHETA1.upper_bound=0
model1.reactions.PHETA1_INB.upper_bound=0
# Serotonin production
model1.reactions.get_by_id('5HLTDL_INB').lower_bound=0.095
### No PAH in the brain
model1.reactions.PHETHPTOX2_INB.lower_bound=0
model1.reactions.PHETHPTOX2_INB.upper_bound=0
model1.reactions.r0399_INB.lower_bound=0
model1.reactions.r0399_INB.upper_bound=0
### Tyrosine hydroxylase
### No Tyr hydroxylase competition
model1.reactions.Phe_TH_INB.upper_bound=0
model1.reactions.PhePde_INB.upper_bound=0
model1.reactions.get_by_id('34dhpheTR_TOBD').upper_bound=0 ##No levodopa from liver to blood.
model1.reactions.Ti1_To1.upper_bound=0.195
model1.solver='glpk'
sol = model1.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.2517557

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'10FTHF5GLUtl': 0.0, '10FTHF5…

In [10]:
model2=model1.copy()

In [11]:
# Kncoking gene '5053.1' to simulate the PKU patients
model2.genes.get_by_id('5053.1').knock_out()
# No phenylpyruvate productionn in the brain and liver
model2.reactions.PHETA1.upper_bound=0.1
model2.reactions.PHETA1_INB.upper_bound=0.1
sol = model2.optimize()
sol.objective_value
fbasol = pfba(model2)
b = show_map(fbasol,map_loc)
b

0.17000014452788614

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'10FTHF5GLUtl': 0.0, '10FTHF5…

In [12]:
model3=model2.copy()

In [13]:
## Only adding tyrosine
model3.reactions.EX_tyr_L_e.lower_bound=-0.1
model3.reactions.PHEPHEr.lower_bound=0
model3.reactions.PHEPHETHRr.lower_bound=0
sol = model3.optimize()
sol.objective_value
fbasol = model3.optimize()
b = show_map(fbasol,map_loc)
b

0.17000014452788542

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'10FTHF5GLUtl': 0.0, '10FTHF5…

In [14]:
## Phe deprivation and Tyr addition
model3.reactions.EX_phe_L_e.upper_bound=-0.04
model3.reactions.EX_phe_L_e.lower_bound=-0.04
sol = model3.optimize()
sol.objective_value
fbasol = model3.optimize()
b = show_map(fbasol,map_loc)
b

0.2517557

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'10FTHF5GLUtl': 0.0, '10FTHF5…

In [15]:
#Only removing the Phe
model4=model2.copy()
model4.reactions.EX_phe_L_e.upper_bound=-0.04
model4.reactions.EX_phe_L_e.lower_bound=-0.04
sol = model4.optimize()
sol.objective_value
fbasol = model4.optimize()
b = show_map(fbasol,map_loc)
b

0.21405345

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'10FTHF5GLUtl': 0.0, '10FTHF5…

In [16]:
model5=model2.copy()
# Taking more Phe for PKU patient to show protein synthesis in the brain
model5.reactions.EX_phe_L_e.lower_bound=-0.21
model5.reactions.EX_phe_L_e.upper_bound=-0.21
sol = model5.optimize()
sol.objective_value
fbasol=pfba(model5)
b = show_map(fbasol,map_loc)
b

0.11218899044381636

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'10FTHF5GLUtl': 0.0, '10FTHF5…